In [ ]:
import pickle
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import copy

sys.path.append(os.path.abspath(os.path.join("..", "src")))
from reconstruct_funcs import *
from utils import *

%load_ext autoreload
%autoreload 2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
with open("../datasets/budha_board/rectification_results/stereo_rectification.pkl", "rb") as f:
    rectification_results = pickle.load(f)

In [3]:
with open("../datasets/budha_board/poses/poses.pkl", "rb") as f:
    poses = pickle.load(f)

In [4]:
with open("../datasets/budha_board/disparities/disparities.pkl", "rb") as f:
    disparities = pickle.load(f)

In [5]:
print(rectification_results.keys())

dict_keys(['R1', 'R2', 'P1', 'P2', 'Q', 'validRoi1', 'validRoi2', 'left_map_x', 'left_map_y', 'right_map_x', 'right_map_y'])


In [6]:
dir = "../datasets/budha_board/rectified_captures"
left_imgs, right_imgs = get_images(dir, prefix="", ext="jpg", print_info=True)

Found 6 left images and 6 right images
First left image: ..\datasets\budha_board\rectified_captures\left_rectified_0.jpg
First right image: ..\datasets\budha_board\rectified_captures\right_rectified_0.jpg


In [7]:
extent = np.array([700, 780, 700])
nbn = 40
std = 2
path1 = "../datasets/budha_board/cloud_points/budha_0_outliers.ply"
path2 = "../datasets/budha_board/cloud_points/budha_0_without_outliers.ply"
scene, pcds = reconstruct(left_imgs, disparities, poses, rectification_results, extent,
                           nbn, std, max_valid=10000, path1=path1, path2=path2)

In [8]:
path = "../datasets/budha_board/cloud_points/budha_scene_unaligned.ply"
o3d.io.write_point_cloud(path, scene)

True

In [9]:
source = pcds[3]
target = pcds[0]
transformation = np.eye(4)
save_path = "../datasets/budha_board/cloud_points/2_budha_clouds_unaligned.ply"
draw_registration_result(source, target, transformation, show=False, save_path=save_path)

In [ ]:
threshold = 2
radius = 0.2
max_nn = 50
trans_init = np.eye(4)
k = 50
base = copy.deepcopy(pcds[0])
base.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius, max_nn=max_nn))
base.orient_normals_consistent_tangent_plane(k=k)
align_pcds = [base]
scene = copy.deepcopy(base) 
transformations = []

source = copy.deepcopy(pcds[3])
print(f"Apply point-to-plane ICP to {3}")
reg_p2l = o3d.pipelines.registration.registration_icp(
    source, base, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPlane(),
    criteria = o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=0,
    relative_fitness=1e-6,
    relative_rmse=1e-6)
)
print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)
source.transform(reg_p2l.transformation)
scene += source
align_pcds.append(source)  
transformations.append(reg_p2l.transformation)


Apply point-to-plane ICP to 3
RegistrationResult with fitness=9.293827e-02, inlier_rmse=1.223151e+00, and correspondence_set size of 66886
Access transformation to get result.
Transformation is:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [11]:
scene_aligned, aligned_pcds, transformations = align_pcds(pcds, np.eye(4), threshold = 2, radius = 0.2, max_nn = 50, k =50)

Apply point-to-plane ICP to 1
RegistrationResult with fitness=4.445487e-01, inlier_rmse=1.012915e+00, and correspondence_set size of 237297
Access transformation to get result.
Transformation is:
[[ 9.99933715e-01 -1.59898305e-05  1.15137446e-02 -5.45537677e-03]
 [-9.19265369e-05  9.99956075e-01  9.37223947e-03 -1.91253503e-02]
 [-1.15133887e-02 -9.37267664e-03  9.99889791e-01 -4.46748957e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Apply point-to-plane ICP to 2
RegistrationResult with fitness=3.782270e-01, inlier_rmse=1.010474e+00, and correspondence_set size of 221256
Access transformation to get result.
Transformation is:
[[ 9.99284455e-01 -1.22316270e-03  3.78032104e-02  4.34324438e-01]
 [ 1.44619658e-04  9.99593211e-01  2.85200368e-02  1.32632611e-01]
 [-3.78227171e-02 -2.84941623e-02  9.98878133e-01  1.52834043e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Apply point-to-plane ICP to 3
RegistrationResult with fitness=2.422403

In [12]:
path = "../datasets/budha_board/cloud_points/budha_scene_aligned.ply"
o3d.io.write_point_cloud(path, scene_aligned)

True

In [ ]:
path = "../datasets/budha_board/cloud_points/budha_scene_aligned.ply"
scene_aligned = o3d.io.read_point_cloud(path)

In [14]:
source = pcds[3]
target = pcds[0]
transformation = transformations[2]
save_path = "../datasets/budha_board/cloud_points/2_budha_clouds_aligned.ply"
draw_registration_result(source, target, transformation, show=False, save_path=save_path)


yellow source 
blue target


In [ ]:
min_bound_budha = np.array([-20, -330, -300])
max_bound_budha = np.array([170, -118, 65])

In [ ]:
box_budha = o3d.geometry.AxisAlignedBoundingBox(min_bound_budha, max_bound_budha)

indices_in_box = box_budha.get_point_indices_within_bounding_box(scene_aligned.points)
pcd_room = scene_aligned.select_by_index(indices_in_box, invert=True)

pcd_budha = scene_aligned.select_by_index(indices_in_box)

In [ ]:
voxel_size = 2
pcd_down = pcd_budha.voxel_down_sample(voxel_size=voxel_size)
mesh_budha = create_mesh_from_point_cloud(pcd_down, voxel_size, max_nn=30, k=30, method="pivoting")
o3d.io.write_triangle_mesh("../datasets/budha_board/meshes/budha_mesh.ply", mesh_budha)

computing mesh using ball pivoting...


True

In [ ]:
voxel_size = 2
pcd_down = pcd_room.voxel_down_sample(voxel_size=voxel_size)
mesh_room = create_mesh_from_point_cloud(pcd_down, voxel_size, max_nn=30, k=30, depth=13, scale=1, linear_fit=False, method="poisson")
o3d.io.write_triangle_mesh("../datasets/budha_board/meshes/room_mesh.ply", mesh_room)

computing mesh using poisson reconstruction...


True

In [19]:
total_mesh = mesh_budha + mesh_room
path = "../datasets/budha_board/meshes/final_mesh.ply"
o3d.io.write_triangle_mesh(path, total_mesh)

True